In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import threading


## Group Assignment
### Team Number: 11
### Team Member Names: Bob Li, Lucas Zaharia, Charles Chen
### Team Strategy Chosen: RISKY

In [2]:
file_name="Tickers.csv"

# Read the File to a pd dataframe
ticker_list=pd.read_csv(file_name,header=None)


In [3]:
# Assign Start and End date for the analysis
start_date="2022-11-02"
end_date= "2022-12-30"

# Determine the minimum number of trading days required to be considered
apple =yf.Ticker("AAPL")
apple_hist = apple.history(start= start_date, end= end_date)
apple_hist.dropna(inplace=True)
min_data_len = len(apple_hist.index)

In [4]:
# Extract first trading day
apple_hist.reset_index(inplace=True)
first_trading_day = str(apple_hist["Date"][0])
first_trading_day = first_trading_day[0:10]

In [5]:
# add_month produces the trading volume in a given month and if the month has at least
# 20 trading days

# add_month: yf.Ticker Str Str -> [Monthly Volume, 0 or 1]
def add_month(ticker, start_here, end_here):
    # Create var called month_num representing if the month is eligible for calculation
    month_num=0
    
    # Get the history of month
    hist=ticker.history(start= start_here, end= end_here)
    
    # get a dataframe of volume
    company=hist["Volume"]
    
    # requires at least 20 trading days to be considered in the average
    if len(company) >= 20:
        # Calculate Sum
        month=company.sum()
        month_num+=1
    else:
        # Or do not add the month in
        month=0
    return [month, month_num]

# Calculate the Monthly Average of Volume of a stock
# Take in a ticker and returns monthly Average of that ticker
def monthly_avg(ticker):
    # Start variable total
    total=0
    
    # Start variable month_num representing the number of months
    month_num=0
    
    # Adding the volume for Jan-Sept
    for i in range(1,9):
        total+=add_month(ticker,f'2022-0{i}-01',f'2022-0{i+1}-01')[0]
        month_num+=add_month(ticker,f'2022-0{i}-01',f'2022-0{i+1}-01')[1]
    
    # Adding the volume for Sept-Oct
    total+=add_month(ticker,f'2022-09-01',f'2022-10-01')[0]
    month_num+=add_month(ticker,f'2022-09-01',f'2022-10-01')[1]
    
    total+=add_month(ticker,f'2022-10-01',f'2022-11-01')[0]
    month_num+=add_month(ticker,f'2022-10-01',f'2022-11-01')[1]
    
    # Calculating the monthly Average
    month_avg=total/month_num
    return month_avg
    

# Take in a dataframe of ticker name and an index (i)
# Function edits the indexed ticker name in the dataframe (remove it when it does not meet requirements)
def check_ticker_valid(i, ticker_list): 
    # Find indexed ticker name
    ticker=ticker_list.iloc[i,0]
    
    # Create the yfinance ticker
    company=yf.Ticker(ticker)
    
    # Use try except method to catch errors
    try:
        # Get the market of the ticker
        location=company.info['market']

        # Call monthly_avg to get the monthly average volume
        avg=monthly_avg(company)
        
        # calculate the length of data range
        company_hist = company.history(start= start_date, end= end_date)
        company_hist.dropna(inplace=True)
        lenn = len(company_hist.index)

        # To be eligible the stock must:
        #  - Be in US market
        #  - Have average monthly volume >=200000
        #  - Not an index
        #  - Has a certain number of trading days within time period (our requirement)
        if location != 'us_market' or avg < 200000 or lenn< min_data_len or "^" == ticker[0]:
            # Remove the Ticker from list if it does not meet requirements
            print("k")
            ticker_list.iloc[i,0]="N/A"
       
    except:
        # Remove the Ticker from list if it does not meet requirements
        company_hist = company.history(start= start_date, end= end_date)
        company_hist.dropna(inplace=True)
        lenn = len(company_hist.index)
        print("l",company.info['market'],monthly_avg(company),lenn)
        ticker_list.iloc[i,0]="N/A"
print(ticker_list)

             0
0         ACRV
1        asdfa
2         AAPL
3         ABBV
4          LOW
5         AUST
6         HOOD
7   INVALIDTIC
8         AMZN
9          AXP
10         BAC
11        BMBL
12          BK
13     HDFC.NS
14          SQ
15        CELG
16          VZ
17       CMCSA
18        SHOP
19        SHOP
20        COST
21        CSCO
22         CVS
23          GM
24        GOOG
25         JPM
26         IBM
27        ORCL
28         OXY
29        DUOL
30         PEP
31        T.TO
32         SLB
33          SO
34         SPG
35        SHOP
36        COST
37         QQQ


In [6]:
# Create a list of threads
threads = []

# Start Threading in a loop
for i in range(0,len(ticker_list)):
    # Create a new thread
    t = threading.Thread(target=check_ticker_valid, args=[i, ticker_list])
    
    # Start thread
    t.start()
    
    # Add thread to thread list
    threads.append(t)

# Stop all threads in list
for thread in threads:
    thread.join()

Exception in thread Thread-17:
Traceback (most recent call last):
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 63, in check_ticker_valid
KeyError: 'market'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bobli/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/bobli/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 88, in check_ticker_valid
KeyError: 'market'
Exception in thread Thread-18:
Traceback (most recent call last):
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 63, in check_ticker_valid
KeyError: 'market'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  Fil

- INVALIDTIC: No data found, symbol may be delisted


Exception in thread Thread-20:
Traceback (most recent call last):
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 63, in check_ticker_valid
KeyError: 'market'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bobli/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/bobli/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 88, in check_ticker_valid
KeyError: 'market'
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 63, in check_ticker_valid
KeyError: 'market'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  Fil

- CELG: No data found, symbol may be delisted


Exception in thread Thread-33:
Traceback (most recent call last):
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 63, in check_ticker_valid
KeyError: 'market'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bobli/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/bobli/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 88, in check_ticker_valid
KeyError: 'market'
Exception in thread Thread-19:
Traceback (most recent call last):
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 63, in check_ticker_valid
KeyError: 'market'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  Fil

- ASDFA: No data found, symbol may be delisted


Exception in thread Thread-28:
Traceback (most recent call last):
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 63, in check_ticker_valid
KeyError: 'market'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bobli/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/bobli/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 88, in check_ticker_valid
KeyError: 'market'
Exception in thread Thread-35:
Traceback (most recent call last):
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 63, in check_ticker_valid
KeyError: 'market'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  Fil

Exception in thread Thread-25:
Traceback (most recent call last):
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 63, in check_ticker_valid
KeyError: 'market'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bobli/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/bobli/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 88, in check_ticker_valid
KeyError: 'market'
Exception in thread Thread-27:
Traceback (most recent call last):
  File "/var/folders/41/qcn7wj1j0qsbvxvf7ssrkn5c0000gn/T/ipykernel_71633/1491118066.py", line 63, in check_ticker_valid
KeyError: 'market'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  Fil

## Note on above code block

### You may see that there are outputs saying that certain tickers may be delisted. This is expected and normal. Yahoo Finance does this automatically. You may also notice that there are multiple delisted messages for the same ticker. Please note that this is not because we are retrieving the same data multiple times from the same ticker. This is because we use threading to split up calculating monthly volume, so when we call Yahoo Finance on each month, we get the same delisted message.

In [7]:
##New Code

In [8]:
ticker_list=ticker_list.drop_duplicates()


In [9]:
# Remove empty entries
ticker_list=ticker_list.replace('N/A',np.NaN)
ticker_list=ticker_list.dropna()

# Reset index
ticker_list.reset_index(drop=True, inplace=True)

In [10]:
# Create empty column called score
ticker_list['score']=''

# Calculates and insert score for each ticker in the dataframe
# Consumes Ticker list dataframe, index, start and end date
def eval_ticker(ticker_list,i,start_date, end_date):
    ticker_name=ticker_list.iloc[i,0]
    ticker=yf.Ticker(ticker_name)
    company=ticker.history(start= start_date, end= end_date)
    company=company["Close"]
    company=company.pct_change()
    company=company.dropna()
    standard_deviation=company.std()
    score=standard_deviation
    ticker_list['score'][i]=score

In [11]:
# Create a list of threads
threads = []

# Start Threading in a loop
for i in range(0,len(ticker_list)):
    # Create a new thread
    t = threading.Thread(target=eval_ticker, args=[ticker_list,i,start_date, end_date])
    
    # Start thread
    t.start()
    
    # Add thread to thread list
    threads.append(t)

# Stop all threads in list
for thread in threads:
    thread.join()


- CELG: No data found, symbol may be delisted
- INVALIDTIC: No data found, symbol may be delisted- ASDFA: No data found, symbol may be delisted



In [12]:
# Sort the dataframe by score in decreasing order
ticker_list.sort_values(by=['score'], ascending=False, inplace=True)

# Reset the index
ticker_list.reset_index(drop=True,inplace=True)
ticker_list.dropna(inplace=True)


In [13]:
# Generates a dictionary where the key is the stock ticker and the associated value
# is a DataFrame containing daily close prices
# Parameters:
#  - ticker_lst (List): List containing stock tickers
# Returns:
#  - df_dict (Dictionary): Dictionary containing stock prices
def generate_stock_df_dict(ticker_lst):
    # Variables
    hist_interval = "1d"  
    df_dict = {}
    
    # Loop through each ticker in ticker_lst
    for ticker in ticker_lst:
        stock = yf.Ticker(ticker)
        
        # Get stock prices
        stock_hist = stock.history(start=start_date, interval=hist_interval)
        
        # Drop empty rows
        stock_hist.dropna(inplace=True)
        
        # Add DF with just Close column to dict
        df_dict[ticker] = stock_hist[["Close"]]
        
    # Return dict
    return df_dict

# Finds correlation in expected returns between two stocks when given a dictionary
# containing their stock prices.
# Parameters:
#  - ticker (String): Ticker of first stock
#  - second_ticker (String): Ticker of second stock
#  - df_dict (Dictionary): Dictionary containing stock prices
# Returns:
#  - returns_corr (Float): Correlation between expected returns
def find_corr(ticker, second_ticker, df_dict):
    # Variables
    ls = "_" + ticker
    rs = "_" + second_ticker
    
    # Join the price DataFrames for the two stocks
    prices = df_dict[ticker].join(df_dict[second_ticker], lsuffix=ls, rsuffix=rs)
    
    # Drop empty rows
    prices.dropna(inplace=True)
    
    # Get daily returns and drop first row
    daily_returns = prices.pct_change()
    daily_returns = daily_returns.iloc[1:]
    
    # Extract correlation
    returns_corr = daily_returns.corr().iloc[0, 1]
    
    # Return correlation
    return returns_corr

# Generates a list of portfolios, where each portfolio is composed of the 11 most
# correlated stocks to each stock in ticker_lst as well as the stock in ticker_lst
# Parameters:
#  - ticker_lst (List): List of tickers with length of at least 12
#  - df_dict (Dictionary): Dictionary containing stock prices
# Returns:
#  - portfolios (2D List): List of portfolios
def generate_portfolios(ticker_lst, df_dict):
    # Create empty list
    portfolios = []
    
    # Loop through each ticker in ticker_lst
    for ticker in ticker_lst:
        # Copy ticker_lst to avoid reference problems
        ticker_lst_copy = ticker_lst.copy()
        
        # Remove current ticker from copy
        ticker_lst_copy.remove(ticker)
        
        # Create a new DF where the index is ticker_lst_copy
        corr_df = pd.DataFrame(index=ticker_lst_copy)
        
        # Name the index Stock
        corr_df.index.name = "Stock"
        
        # Create a new column called Corr where each value is initially -2
        corr_df["Corr"] = -2
        
        # Loop through each ticker in ticker_lst_copy
        for second_ticker in ticker_lst_copy:
            # Find correlation between tickers in outer and inner for loops
            corr = find_corr(ticker, second_ticker, df_dict)
            
            # Place correlation value in appropriate location in DF
            corr_df.loc[second_ticker, "Corr"] = corr
        
        # Reset index
        corr_df.reset_index(inplace=True)
        
        # Sort DF values by Corr column from most correlated to least correlated
        corr_df.sort_values(by="Corr", ascending=False, inplace=True)
        
        # Create a new one element list containing ticker in outer for loop
        current_portfolio = [ticker]
        
        # Extract 11 most correlated stocks as a list
        additional_stocks = list(corr_df.head(11).Stock)
        
        # Extend the one element list to have the correlated stocks
        current_portfolio.extend(additional_stocks)
        
        # Append current portfolio to list of portfolios
        portfolios.append(current_portfolio)
    
    # Return list of portfolios
    return portfolios



          0     score
0      HOOD  0.058023
1        SQ  0.053289
2      AUST  0.048417
3      SHOP  0.047617
4      ACRV  0.046851
5      DUOL  0.038016
6      BMBL  0.037746
7      AMZN  0.029796
8       OXY  0.026929
9      GOOG  0.024964
10      SLB  0.024855
11     AAPL  0.024827
12      LOW  0.023663
13       GM  0.022772
14      QQQ  0.019815
15      SPG  0.018923
16      AXP  0.018719
17     COST  0.017605
18    CMCSA  0.017386
19      BAC  0.016861
20       BK  0.016114
21     CSCO  0.015585
22     ORCL  0.014098
23      IBM   0.01394
24       SO  0.013313
25      CVS  0.013216
26      JPM  0.013046
27  HDFC.NS  0.012926
28       VZ  0.012537
29     ABBV  0.009196
30     T.TO  0.008991
31      PEP  0.008652


In [15]:
# Keep 25 most risky stocks
ticker_list = ticker_list.head(25)

# Convert tickers into list
risky_tickers = list(ticker_list[0])

# Generate dictionary of stock prices
stock_price_dict = generate_stock_df_dict(risky_tickers)

# Generate 25 risky portfolios
portfolios = generate_portfolios(risky_tickers, stock_price_dict)

In [16]:

print(portfolios,risky_tickers, stock_price_dict,ticker_list)

[['HOOD', 'BMBL', 'SHOP', 'OXY', 'SQ', 'AMZN', 'QQQ', 'SPG', 'GM', 'AAPL', 'CMCSA', 'GOOG'], ['SQ', 'QQQ', 'GOOG', 'AMZN', 'AXP', 'AAPL', 'SHOP', 'IBM', 'CMCSA', 'BK', 'GM', 'LOW'], ['AUST', 'CMCSA', 'AXP', 'ACRV', 'BAC', 'LOW', 'GM', 'SHOP', 'GOOG', 'SQ', 'OXY', 'SLB'], ['SHOP', 'QQQ', 'AMZN', 'SQ', 'AAPL', 'BMBL', 'GOOG', 'HOOD', 'LOW', 'BK', 'CMCSA', 'BAC'], ['ACRV', 'GOOG', 'AMZN', 'SQ', 'QQQ', 'SHOP', 'AAPL', 'BMBL', 'IBM', 'HOOD', 'LOW', 'AXP'], ['DUOL', 'SPG', 'SO', 'QQQ', 'AXP', 'AMZN', 'AAPL', 'CSCO', 'SQ', 'GM', 'SHOP', 'BK'], ['BMBL', 'AMZN', 'QQQ', 'SHOP', 'GOOG', 'AAPL', 'SQ', 'HOOD', 'GM', 'LOW', 'SPG', 'CMCSA'], ['AMZN', 'QQQ', 'GOOG', 'SQ', 'AAPL', 'BMBL', 'SHOP', 'LOW', 'AXP', 'CMCSA', 'BK', 'BAC'], ['OXY', 'SLB', 'GM', 'HOOD', 'SQ', 'CMCSA', 'QQQ', 'SPG', 'AAPL', 'AXP', 'IBM', 'AMZN'], ['GOOG', 'QQQ', 'AMZN', 'SQ', 'AAPL', 'LOW', 'AXP', 'IBM', 'SHOP', 'CMCSA', 'BMBL', 'BK'], ['SLB', 'OXY', 'GM', 'IBM', 'SPG', 'AXP', 'SQ', 'CMCSA', 'QQQ', 'AAPL', 'HOOD', 'BAC'], ['AAPL

In [17]:
# Stores portfolio standard deviations
portfolio_stds = []

# Builds a portfolio with $500000 and adds the standard deviation of the portfolio to portfolio_stds
# Parameters: A list of 12 tickers
def buildportfolio(lst):
    portfolio = pd.DataFrame()
    
    #Determines how much of the portfolio the stock will be worth, based on its position in the list
    for x in range(len(lst)):
        if x <= 1:
            investment = 125000
        elif x == 2:
            investment = 62500
        else:
            investment = 62500/3
        
        #Gets closing price value from the dictionary defined earlier
        portfolio[lst[x]] = stock_price_dict[lst[x]]
        
        #Defines the number of shares based on the stock price from the first day
        print(portfolio)
        try:
            num_shares = investment / portfolio.loc[first_trading_day, lst[x]]
        except:
            num_shares = investment / portfolio[lst[x]].iloc[0]
        
        #Multiplies closing value by number of shares 
        portfolio[lst[x]] = portfolio[lst[x]] * num_shares
        
    #Drops NaN values
    portfolio = portfolio.dropna()
    
    #Adds all stocks into a final column to track the portfolio value over time
    portfolio["portfolio value"] = portfolio.sum(axis = 1)
    
    #Adds the standard deviation of the portfolio to the portfolio_stds
    portfolio_stds.append(portfolio['portfolio value'].std())
    
#Applys buildportfolio to every list of tickers in portfolios
for n in portfolios:
    buildportfolio(n)

             HOOD
Date             
2022-11-02  11.40
2022-11-03  12.33
2022-11-04  12.28
2022-11-07  12.03
2022-11-08   9.74
2022-11-09   8.40
2022-11-10   9.27
2022-11-11  10.47
2022-11-14   9.56
2022-11-15  10.15
2022-11-16   9.32
2022-11-17   9.47
2022-11-18   9.19
2022-11-21   8.85
2022-11-22   9.09
2022-11-23   9.46
2022-11-25   9.39
2022-11-28   9.19
2022-11-29   9.18
2022-11-30   9.59
2022-12-01   9.87
2022-12-02   9.99
2022-12-05   9.67
2022-12-06   9.39
2022-12-07   9.26
2022-12-08   9.52
2022-12-09   9.31
2022-12-12   9.58
2022-12-13   9.35
2022-12-14   9.09
2022-12-15   8.67
2022-12-16   8.61
2022-12-19   8.20
2022-12-20   8.10
2022-12-21   8.17
2022-12-22   7.99
2022-12-23   7.95
2022-12-27   7.70
2022-12-28   7.70
2022-12-29   8.05
2022-12-30   8.14
2023-01-03   8.08
2023-01-04   8.36
2023-01-05   8.11
2023-01-06   8.25
2023-01-09   8.55
2023-01-10   8.72
2023-01-11   8.76
2023-01-12   9.04
2023-01-13   9.32
                     HOOD       BMBL
Date                       

2023-01-13  20227.172313  20245.085241  134.759995  
                     ACRV           GOOG          AMZN            SQ  \
Date                                                                   
2022-11-15  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-16  125826.326731  125341.871758  61350.313542  19745.958724   
2022-11-17  118088.948645  124721.432818  59916.361010  19303.057261   
2022-11-18  118539.665804  123835.091474  59467.857456  18686.972540   
2022-11-21   93975.365743  121340.661272  58406.608047  17559.848502   
2022-11-22   94125.602408  123239.972432  58874.061709  17806.851199   
2022-11-23   93900.243829  125126.618812  59461.539147  18147.542632   
2022-11-25   93900.243829  123581.844190  59006.722103  17994.231921   
2022-11-28   93900.243829  121872.466078  59347.833681  17423.571713   
2022-11-29   93900.243829  120846.841143  58381.339630  17633.665774   
2022-11-30   96529.453522  128456.740901  60983.928728  19240.596660   
2022-12-01 

2023-01-13  27318.967456  36.509998  
                     DUOL            SPG            SO           QQQ  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  128849.041030  126086.373550  62548.220891  20426.358575   
2022-11-04  124921.762655  129435.085412  62606.096263  20754.918195   
2022-11-07  121870.697185  130891.051549  61111.108822  20983.106534   
2022-11-08  122840.785355  129267.088005  61680.167584  21136.017011   
2022-11-09  120478.158387  126467.159698  61188.267399  20648.274756   
2022-11-10  132604.206796  134441.358591  63377.696749  22171.880316   
2022-11-11  114892.353339  134620.549560  62779.700301  22580.422516   
2022-11-14  114923.641115  131630.228179  62249.226346  22382.817229   
2022-11-15  115502.566433  134575.758226  63551.308147  22692.557982   
2022-11-16  110558.274193  129457.481079  63493.440134  22382.817229   
2022-11-17  108383.397754 

2023-01-13  133226.144736  280.970001
                     GOOG            QQQ        AMZN
Date                                                
2022-11-02  125000.000000  125000.000000   92.120003
2022-11-03  119860.454457  122558.151449   89.300003
2022-11-04  124468.814247  124529.509171   90.980003
2022-11-07  127268.292518  125898.639206   90.529999
2022-11-08  127641.558478  126816.102066   89.980003
2022-11-09  125473.759378  123889.648536   86.139999
2022-11-10  135192.946048  133031.281897   96.629997
2022-11-11  138868.157368  135482.535096  100.790001
2022-11-14  137863.212237  134296.903375   98.489998
2022-11-15  141725.051061  136155.347893   98.940002
2022-11-16  142112.665400  134296.903375   97.120003
2022-11-17  141409.211475  134000.491856   94.849998
2022-11-18  140404.277298  134005.201359   94.139999
2022-11-21  137576.091316  132626.666676   92.459999
2022-11-22  139729.531084  134536.858293   93.199997
2022-11-23  141868.611521  135887.164675   94.129997
2022-11-

2023-01-13  23439.133106  22613.239621  71.650002  
                      SLB            OXY            GM           IBM  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  126613.218262  126863.581594  62483.771197  20474.006587   
2022-11-04  129790.768253  128815.046089  63278.815447  20853.127494   
2022-11-07  133970.478166  133561.889682  63911.602612  21063.240652   
2022-11-08  133677.158974  131557.671303  63359.940896  21322.077184   
2022-11-09  129717.443117  119426.866672  61753.635970  20918.596913   
2022-11-10  129790.768253  123909.994496  64479.493131  21503.263453   
2022-11-11  133994.916770  130678.627683  66718.585221  21798.642515   
2022-11-14  132161.713776  128832.630717  64787.772311  21955.467097   
2022-11-15  133628.272442  130854.433725  65290.759995  21976.782992   
2022-11-16  129521.906313  126459.215627  62418.874551  22004.190470   
2022-11-17  

...

2023-01-13  140634.798638  135679.437724  71.650002
                      SPG            AXP            SQ         QQQ
Date                                                              
2022-11-02  125000.000000  125000.000000  62500.000000  265.679993
2022-11-03  126086.373550  121350.172089  61664.989533  260.489990
2022-11-04  129435.085412  126062.718833  68756.864570  264.679993
2022-11-07  130891.051549  128031.355163  71685.119497  267.589996
2022-11-08  129267.088005  130836.234275  71502.103564  269.540009
2022-11-09  126467.159698  127055.749661  65439.695877  263.320007
2022-11-10  134441.358591  134799.651568  77095.537014  282.750000
2022-11-11  134620.549560  134921.602256  81968.343318  287.959991
2022-11-14  131630.228179  134259.586135  80378.382856  285.440002
2022-11-15  134575.758226  134050.522116  83935.759143  289.390015
2022-11-16  129457.481079  132900.703244  79554.817703  285.440002
2022-11-17  129681.480470  131219.511663  77770.404721  284.809998
2022-11-18

2023-01-13  19358.063223  22613.239621  20336.888908  22190.259042   92.800003  
                    CMCSA            QQQ           IBM            SQ  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  122856.677898  122558.151449  61422.019760  20554.996511   
2022-11-04  125363.960502  124529.509171  62559.382482  22918.954857   
2022-11-07  127790.361281  125898.639206  63189.721957  23895.039832   
2022-11-08  129003.557813  126816.102066  63966.231552  23834.034521   
2022-11-09  126941.117537  123889.648536  62755.790740  21813.231959   
2022-11-10  132521.840098  133031.281897  64509.790359  25698.512338   
2022-11-11  137576.838506  135482.535096  65395.927544  27322.781106   
2022-11-14  138021.680834  134296.903375  65866.401291  26792.794285   
2022-11-15  138628.271387  136155.347893  65930.348976  27978.586381   
2022-11-16  137415.074854  134296.903375  66012.571411 

2023-01-13  22204.357456  23800.548794  89.199997  
                      BAC             BK         CMCSA           QQQ  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  124307.284698  123483.580741  61428.338949  20426.358575   
2022-11-04  127424.496952  126338.013657  62681.980251  20754.918195   
2022-11-07  128186.471893  126129.868117  63895.180640  20983.106534   
2022-11-08  128532.836151  126962.416250  64501.778907  21136.017011   
2022-11-09  126489.333276  125178.394259  63470.558769  20648.274756   
2022-11-10  132065.672302  131035.914412  66260.920049  22171.880316   
2022-11-11  133035.465797  132373.928069  68788.419253  22580.422516   
2022-11-14  130784.144368  129578.975030  69010.840417  22382.817229   
2022-11-15  130576.339026  130322.314690  69314.135693  22692.557982   
2022-11-16  129398.726976  130262.846156  68707.537427  22382.817229   
2022-11-17  

2023-01-13  26238.811939  27318.967456  22204.357456  48.880001  
                      IBM            AXP           QQQ          AAPL  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  122844.039521  121350.172089  61279.075724  19949.896293   
2022-11-04  125118.764965  126062.718833  62264.754586  19878.072072   
2022-11-07  126379.443915  128031.355163  62949.319603  19955.641266   
2022-11-08  127932.463104  130836.234275  63408.051033  20038.957626   
2022-11-09  125511.581480  127055.749661  61944.824268  19373.864639   
2022-11-10  129019.580718  134799.651568  66515.640948  21097.645940   
2022-11-11  130791.855088  134921.602256  67741.267548  21504.171293   
2022-11-14  131732.802582  134259.586135  67148.451688  21300.190769   
2022-11-15  131860.697952  134050.522116  68077.673947  21553.011237   
2022-11-16  132025.142822  132900.703244  67148.451688  21373.450685  

2023-01-13  19358.063223  20908.305507  22190.259042  22212.781894  79.360001  
                       SO            AXP           QQQ           SPG  \
Date                                                                   
2022-11-02  125000.000000  125000.000000  62500.000000  20833.333333   
2022-11-03  125096.441782  121350.172089  61279.075724  21014.395592   
2022-11-04  125212.192525  126062.718833  62264.754586  21572.514235   
2022-11-07  122222.217644  128031.355163  62949.319603  21815.175258   
2022-11-08  123360.335168  130836.234275  63408.051033  21544.514667   
2022-11-09  122376.534797  127055.749661  61944.824268  21077.859950   
2022-11-10  126755.393499  134799.651568  66515.640948  22406.893099   
2022-11-11  125559.400603  134921.602256  67741.267548  22436.758260   
2022-11-14  124498.452693  134259.586135  67148.451688  21938.371363   
2022-11-15  127102.616294  134050.522116  68077.673947  22429.293038   
2022-11-16  126986.880268  132900.703244  67148.451688  

In [18]:
#Gets the list of tickers from portfolios based on the position of the largest standard deviation in portfolio_stds, since order is preserved
finalport_tickerslst = portfolios[portfolio_stds.index(max(portfolio_stds))]

finalport_pricelst = []

# Defines finalport_pricelst as the price of each ticker at the required date
for i in range(len(finalport_tickerslst)):
    finalport_pricelst.append(stock_price_dict[finalport_tickerslst[i]].loc["2022-11-25", "Close"])
    
finalport_shareslst = []

#Defines finalport_shareslst as the number of shares of each stock, based on finalport_pricelst and the position of the ticker in the finalport_tickerslst
for n in range(len(finalport_tickerslst)):
    value = -1
    if n <= 1:
        value = 125000
    elif n == 2:
        value = 62500
    else:
        value = 62500/3
    finalport_shareslst.append(value/finalport_pricelst[n])
    
#Creates a dictionary with the data above
finalport = {"Ticker" : finalport_tickerslst,
             "Price" : finalport_pricelst,
             "Shares" : finalport_shareslst}

#Creates the index for the portfolio
index_list = list(range(1, 13))

#Creates the dataframe from the data in finalport
Portfolio_Final = pd.DataFrame(finalport, index=index_list)

#Adds a column for Value using price * shares
Portfolio_Final["Value"] = Portfolio_Final.Price * Portfolio_Final.Shares

#Adds a column for Weight based oln the value of each stock
Portfolio_Final["Weight"] = Portfolio_Final.Value / 500000 * 100

Portfolio_Final

,Ticker,Price,Shares,Value,Weight
1,HOOD,9.390000,13312.033592,125000.000000,25.000000
2,BMBL,22.049999,5668.934437,125000.000000,25.000000
3,SHOP,36.790001,1698.831162,62500.000000,12.500000
4,OXY,70.279999,296.433320,20833.333333,4.166667
5,SQ,63.380001,328.705159,20833.333333,4.166667
6,AMZN,93.410004,223.031073,20833.333333,4.166667
7,QQQ,286.920013,72.610248,20833.333333,4.166667
8,SPG,120.699997,172.604257,20833.333333,4.166667
9,GM,40.459999,514.911859,20833.333333,4.166667
10,AAPL,148.110001,140.661220,20833.333333,4.166667


In [19]:
print("Total Value:", Portfolio_Final.Value.sum())
print("Total Weight:", Portfolio_Final.Weight.sum())

Total Value: 499999.9999999999
Total Weight: 100.00000000000003


In [20]:
#Creates another dataframe using only the Ticker and Shares column from Portfolio_Final
Stocks_Final = Portfolio_Final[["Ticker", "Shares"]]
print(Stocks_Final)
#Exports the dataframe to a csv file
Stocks_Final.to_csv('Stocks_Group_11.csv')



   Ticker        Shares
1    HOOD  13312.033592
2    BMBL   5668.934437
3    SHOP   1698.831162
4     OXY    296.433320
5      SQ    328.705159
6    AMZN    223.031073
7     QQQ     72.610248
8     SPG    172.604257
9      GM    514.911859
10   AAPL    140.661220
11  CMCSA    584.385202
12   GOOG    213.456287


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Bob Li  
Lucas Zaharia  
Charles Chen